In [3]:
import torch

### 1. Load pre-trained model (adapter + roberta) and fine-tuned model
- Here model_path is the path of your pre-trained adapter model (the name of checkpoint would be pytorch_model.bin)
- model_path2 is the path of your fine-tuned adapter model by executing train.py in don't stop repo (the name of checkpoint would be best.th)

In [6]:

### THESE WERE BOTH MY CHEMPROT MODELS

## load model with weights of pre-trained model (pre-trained adapter + Roberta)

model_path = './pytorch_model.bin'### Here plug in pretrained adapter model checkpoint path
model_adapter = torch.load(model_path, map_location=torch.device('cpu'))

## load model with weights of fine-tuned model (model after fine-tuning pre-trained (adapter + Roberta))

model_path2 = './best.th' ### Here plug in fine-tuned pre-trained (adapter + Roberta) model checkpoint path
model_ft = torch.load(model_path2, map_location=torch.device('cpu'))

- Print name of layers

In [7]:
## printing keys for each model
## pre-trained model
for key in model_adapter:
    print('keys of pre-trained: ', key)

ttention.self.value.weight
keys of pre-trained:  roberta.encoder.layer.0.attention.self.value.bias
keys of pre-trained:  roberta.encoder.layer.0.attention.output.dense.weight
keys of pre-trained:  roberta.encoder.layer.0.attention.output.dense.bias
keys of pre-trained:  roberta.encoder.layer.0.attention.output.LayerNorm.weight
keys of pre-trained:  roberta.encoder.layer.0.attention.output.LayerNorm.bias
keys of pre-trained:  roberta.encoder.layer.0.intermediate.dense.weight
keys of pre-trained:  roberta.encoder.layer.0.intermediate.dense.bias
keys of pre-trained:  roberta.encoder.layer.0.output.dense.weight
keys of pre-trained:  roberta.encoder.layer.0.output.dense.bias
keys of pre-trained:  roberta.encoder.layer.0.output.LayerNorm.weight
keys of pre-trained:  roberta.encoder.layer.0.output.LayerNorm.bias
keys of pre-trained:  roberta.encoder.layer.0.output.layer_text_lang_adapters.nli/scitail@ukp.adapter_down.0.weight
keys of pre-trained:  roberta.encoder.layer.0.output.layer_text_lan

- The fine tuning script and the train script use different modules (pytorch vs AllenNLP)
- To compare, we need to cut out the front part of the module names

In [8]:
model_truncated_adapters = [i.split('roberta.')[-1] for i in list(model_adapter.keys())]

In [13]:
model_truncated_adapters[:10]

['embeddings.word_embeddings.weight',
 'embeddings.position_embeddings.weight',
 'embeddings.token_type_embeddings.weight',
 'embeddings.LayerNorm.weight',
 'embeddings.LayerNorm.bias',
 'encoder.layer.0.attention.self.query.weight',
 'encoder.layer.0.attention.self.query.bias',
 'encoder.layer.0.attention.self.key.weight',
 'encoder.layer.0.attention.self.key.bias',
 'encoder.layer.0.attention.self.value.weight']

In [10]:
 truncated_model_ft = [i.split('_text_field_embedder.token_embedder_roberta.transformer_model.')[-1] for i in list(model_ft.keys())]

In [11]:
truncated_model_ft[:10]

['embeddings.word_embeddings.weight',
 'embeddings.position_embeddings.weight',
 'embeddings.token_type_embeddings.weight',
 'embeddings.LayerNorm.weight',
 'embeddings.LayerNorm.bias',
 'encoder.layer.0.attention.self.query.weight',
 'encoder.layer.0.attention.self.query.bias',
 'encoder.layer.0.attention.self.key.weight',
 'encoder.layer.0.attention.self.key.bias',
 'encoder.layer.0.attention.self.value.weight']

- Now, we can compare weights

In [17]:
## test 1
diff_sum = 0
test_3 = False
for mt_key, mt_og_key in zip(model_truncated_adapters, list(model_adapter.keys())):
    for ft_key, ft_og_key in zip(truncated_model_ft, list(model_ft.keys())):
        if mt_key == ft_key:
            print(f"mt_og_key {mt_og_key}")
            print(f"ft_og_key {ft_og_key}")
            diff_sum += torch.abs(torch.sum(model_adapter[mt_og_key] - model_ft[ft_og_key])).cpu().numpy()
            print('diff_sum', diff_sum)
            print('')
    else:
        continue


.encoder.layer.5.output.LayerNorm.bias
ft_og_key _text_field_embedder.token_embedder_roberta.transformer_model.encoder.layer.5.output.LayerNorm.bias
diff_sum 2819.7550011882363

mt_og_key roberta.encoder.layer.6.attention.self.query.weight
ft_og_key _text_field_embedder.token_embedder_roberta.transformer_model.encoder.layer.6.attention.self.query.weight
diff_sum 2820.636615515667

mt_og_key roberta.encoder.layer.6.attention.self.query.bias
ft_og_key _text_field_embedder.token_embedder_roberta.transformer_model.encoder.layer.6.attention.self.query.bias
diff_sum 2820.6542719127415

mt_og_key roberta.encoder.layer.6.attention.self.key.weight
ft_og_key _text_field_embedder.token_embedder_roberta.transformer_model.encoder.layer.6.attention.self.key.weight
diff_sum 2820.685570028591

mt_og_key roberta.encoder.layer.6.attention.self.key.bias
ft_og_key _text_field_embedder.token_embedder_roberta.transformer_model.encoder.layer.6.attention.self.key.bias
diff_sum 2820.6855823172814

mt_og_key ro